This notebook runs a defined pipeline on HCP subjects, on the Cedars-Sinai cluster.
After it is done preprocessing all subjects, it runs the leave-one-out IQ prediction a la Finn.

# Load libraries & helper functions

In [1]:
from helpers import *
%matplotlib inline

# Declare parameters

In [2]:
config.DATADIR                 = '/scratch/duboisjx/data/HCP/MRI'
# fMRI runs
session                        = 'REST1'
fmriRuns                       = ['rfMRI_'+session+'_LR','rfMRI_'+session+'_RL']
# use volume or surface data
config.fmriFileTemplate        = '#fMRIrun#.nii.gz'
#config.fmriFileTemplate        = '#fMRIrun#_Atlas.dtseries.nii'
#config.fmriFileTemplate        = '#fMRIrun#_Atlas_MSMAll.dtseries.nii'
config.melodicFolder           = op.join('#fMRIrun#_hp2000.ica','filtered_func_data.ica') #the code #fMRIrun# will be replaced
config.movementRelativeRMSFile = 'Movement_RelativeRMS.txt'
config.movementRegressorsFile  = 'Movement_Regressors_dt.txt'
# parcellation for FC matrix
config.parcellationName        = 'shen2013'
config.parcellationFile        = '/scratch/duboisjx/data/parcellations/shenetal_neuroimage2013_new/shen_2mm_268_parcellation.nii.gz'
config.nParcels                = 268
config.maskParcelswithAll      = True
config.maskParcelswithGM       = False
# subject selection parameters
config.behavFile               = '/scratch/duboisjx/data/HCP/neuropsych/unrestricted_luckydjuju_6_7_2017_0_44_41.csv'
config.release                 = 'all'
config.outScore                = 'PMAT24_A_CR'
# submit jobs with sge
config.queue        = True

# do not alter the following lines
if config.maskParcelswithGM:
    config.parcellationName = config.parcellationName + '_GM'
tmp = config.fmriFileTemplate.split('.')
if tmp[1]=='nii':
    config.isCifti = False
elif tmp[1]=='dtseries':
    config.isCifti = True
else:
    print 'unknown file extension'

config.preWhitening            = False

# whether to use memmapping (which involves unzipping)
# deprecated
config.useMemMap               = False


# Do work

### preprocess everybody

In [3]:
config.overwrite               = True
config.pipelineName            = 'Gordon1'
config.Operations              = config.operationDict[config.pipelineName]
if config.pipelineName in ['SiegelA','SiegelB']:
    config.useFIX              = True
else:
    config.useFIX              = False

In [5]:
config.scriptlist = list()

config.subject='109325'
iRun = 0
for config.fmriRun in fmriRuns:
    runPipelinePar(launchSubproc=False)
    iRun=iRun+1

# launch array job (if there is something to do)
if len(config.scriptlist)>0:
    if len(config.scriptlist)<10:
        print config.scriptlist
    config.sgeopts      = '-l mem_free=25G -pe smp 6' 
    JobID = fnSubmitJobArrayFromJobList()
    print 'Running array job {} ({} sub jobs)'.format(JobID.split('.')[0],JobID.split('.')[1].split('-')[1].split(':')[0])
    config.joblist.append(JobID.split('.')[0])
    checkProgress(pause=60,verbose=False)

['/scratch/duboisjx/data/HCP/MRI/109325/MNINonLinear/Results/rfMRI_REST1_LR/jobs/s109325_rfMRI_REST1_LR_Gordon1_ciftiFalse_0829113359687431.sh', '/scratch/duboisjx/data/HCP/MRI/109325/MNINonLinear/Results/rfMRI_REST1_RL/jobs/s109325_rfMRI_REST1_RL_Gordon1_ciftiFalse_0829113359867000.sh']
Running array job 541726 (2 sub jobs)
